In [1]:
import os
import numpy as np
import dask.dataframe as dd
import pandas as pd
import concurrent.futures
import warnings
import timeit
import time
warnings.filterwarnings("ignore")
import re
import matching
from geopy import distance

In [19]:
dff = pd.DataFrame()

In [2]:
dff = pd.read_csv('all_base_transport.csv')

In [3]:
dff

,Unnamed: 0,id,code,name,line,type,long,lat
0,0,1,'100053338','Bedminster Road',NaN,'Bus',-2.587660,51.455793
1,1,2,'100053342','Henry St Green Street',NaN,'Bus',-2.587675,51.455784
2,2,3,'100053328','Ilchester Crescent',NaN,'Bus',-2.587791,51.455873
3,3,4,'100053329','Ilchester Crescent',NaN,'Bus',-2.587761,51.455811
4,4,5,'100053324','Swiss Road',NaN,'Bus',-2.587602,51.455721
...,...,...,...,...,...,...,...,...
423047,423048,423049,'8A26AB0D-D980-48B0-A2C1-314600566755','M20 J10A','M20','Motorway',0.920421,51.133086
423048,423049,423050,'58F564B3-CEE8-4C7A-9B6D-FB82097DC68C','M20 J11','M20','Motorway',1.050151,51.095487
423049,423050,423051,'F0D6AAC7-6AA9-4F21-A7B9-76253E91FF33','M20 J11A','M20','Motorway',1.099786,51.093398
423050,423051,423052,'B69471D9-B06A-4664-9911-EB1C6191AD93','M20 J12','M20','Motorway',1.131499,51.091885


In [252]:
df = pd.read_excel('20210702_pb_data_postcode_1.xlsx')

In [3]:
df = pd.read_csv('ZP_RM_61k_with_address_base_ofcom_schools_added.csv')

In [4]:
df

,Unnamed: 0,Unnamed: 0.1,Date,Property type,Price,Source,Address,KeyFeatures,latitude,longitude,...,ufbb_availability,school_dist_0,school_URN_0,school_Establishment_Name_0,school_dist_1,school_URN_1,school_Establishment_Name_1,school_dist_2,school_URN_2,school_Establishment_Name_2
0,0,0,25 Jun 2021,"Detached, Freehold","£995,000",Land Registry,"41 Runcie Close, St Albans, Hertfordshire AL4 9AX",Detached\nFour Bedrooms\nTwo Bathrooms\nStudy\...,51.770175,-0.313172,...,100,0.143433,146878,Wheatfields Junior Mixed School,0.143433,146893,Wheatfields Infants' and Nursery School,0.143433,117212,Wheatfields Junior Mixed School
1,1,1,10 Jun 2021,"Terraced, Freehold","£650,000",Land Registry,"37 Albany Road, Birmingham, West Midlands B17 9JX",VICTORIAN MID TERRACE\nFOUR BEDROOMS\nPERIOD F...,52.459072,-1.950948,...,90,0.138282,132261,Harborne Primary School,0.138282,103212,Harborne Infant School,0.138282,103211,Harborne Junior School
2,2,2,30 Jun 2021,"Detached, Freehold","£350,000",Land Registry,"19 Bibsworth Avenue, Birmingham, West Midlands...",Detached\nThree Bedrooms\nKitchen Diner\nTwo R...,52.432568,-1.859492,...,100,0.602562,103462,St Bernard's Catholic Primary School,0.957317,103519,Moseley School and Sixth Form,0.995820,103167,Billesley Infant School
3,3,3,18 Jun 2021,"Detached, Freehold","£382,500",Land Registry,"24 Sandy Croft, Birmingham, West Midlands B13 0EP",3 Bedrooms\nPorch\nEntrance Hall\nLounge\nKitc...,52.427717,-1.863037,...,100,0.473623,103167,Billesley Infant School,0.473623,138888,Billesley Primary School,0.473623,131942,Billesley Primary School
4,4,4,30 Jul 2021,"Semi-Detached, Freehold","£213,500",Land Registry,"15 Poppy Walk, Hatfield, Hertfordshire AL10 9FP",One Bedroomed House\nAllocated Parking\nChain ...,51.774284,-0.237790,...,0,0.276724,117105,Green Lanes Primary School,0.912132,132105,Birchwood Avenue Primary School,0.912132,117104,Gascoyne Cecil Junior School
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61118,61118,61118,Aug 2021,NaN,"£225,000",NaN,53 Strathmore Avenue LU1 3NY,"Terraced house, Freehold, 3 Beds, 2 Baths, 2 R...",51.872773,-0.408768,...,100,NaN,141014,Education & Youth Services Ltd (Luton),0.342038,134525,Avenue Centre for Education,0.449903,145872,Chiltern Academy
61119,61119,61119,Aug 2021,NaN,"£60,000",NaN,13 Robinson Street DH8 5TR,"Terraced house, Freehold, 2 Beds, 1 Bath, 2 Re...",54.861234,-1.841458,...,100,NaN,114051,Consett Junior School,0.303556,114295,Consett Community Sports College,0.501237,132886,Benfieldside Junior School
61120,61120,61120,Aug 2021,NaN,"£125,000",NaN,10 Merrington Road TF2 8PY,"Semi-detached house, Freehold",52.725638,-2.419358,...,100,0.583646,123453,Muxton Primary School,1.422642,123364,Donnington Wood Infant School and Nursery Centre,1.461241,123557,St Matthew's Church of England Aided Primary S...
61121,61121,61121,Aug 2021,NaN,"£318,000",NaN,"8 Mount View, Church Lane West GU11 3LN",NaN,51.243067,-0.767056,...,92,0.309190,137422,St Joseph's Catholic Primary School,0.309190,116484,St Joseph's Catholic Primary School,0.542398,115836,Rowhill School


In [5]:
from math import sin, cos, sqrt, atan2, radians
def my_dist(coords_1,coords_2):
    try:
        R = 6373.0

        lat1 = radians(coords_1[0])
        lon1 = radians(coords_1[1])
        lat2 = radians(coords_2[0])
        lon2 = radians(coords_2[1])

        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))

        distance = R * c

        return distance
    except:
        return 'Algo Not Found'

In [6]:
a = list(dff['lat'].astype(float))
b = list(dff['long'].astype(float))
c = []
for x in range(len(a)):
    c.append((a[x],b[x]))

In [7]:
s1 = list(c)

In [8]:
from scipy import spatial
tree = spatial.KDTree(s1)


In [9]:
x = 100
res1 = tree.query([(df.iloc[x]['LATITUDE'],df.iloc[x]['LONGITUDE'])],k=3)

In [10]:
res1

(array([[0.00102503, 0.00118188, 0.0012211 ]]),
 array([[273399, 273439, 273437]]))

In [14]:
res1[1][0].tolist()

[334927, 334928, 323291]

In [15]:
res1[0][0].tolist()

[0.001504080473727412, 0.0018441034782634676, 0.002616214278902734]

In [28]:
df['transport_code_0'] = '-'
df['transport_name_0'] = '-'
df['transport_type_0'] = '-'
df['transport_dist_0'] = '-'
df['transport_code_1'] = '-'
df['transport_name_1'] = '-'
df['transport_type_1'] = '-'
df['transport_dist_1'] = '-'
df['transport_code_2'] = '-'
df['transport_name_2'] = '-'
df['transport_type_2'] = '-'
df['transport_dist_2'] = '-'

In [23]:
my_dist((df.iloc[0]['LATITUDE'],df.iloc[0]['LONGITUDE']),(dff.iloc[0]['lat'],dff.iloc[0]['long']))

137.4866750187595

In [25]:
(df.iloc[0]['LATITUDE'],df.iloc[0]['LONGITUDE']),(dff.iloc[0]['lat'],dff.iloc[0]['long'])

((52.0300681, -0.8198556), (51.455793045200004, -2.5876604366))

In [16]:
dff

,Unnamed: 0,id,code,name,line,type,long,lat
0,0,1,'100053338','Bedminster Road',NaN,'Bus',-2.587660,51.455793
1,1,2,'100053342','Henry St Green Street',NaN,'Bus',-2.587675,51.455784
2,2,3,'100053328','Ilchester Crescent',NaN,'Bus',-2.587791,51.455873
3,3,4,'100053329','Ilchester Crescent',NaN,'Bus',-2.587761,51.455811
4,4,5,'100053324','Swiss Road',NaN,'Bus',-2.587602,51.455721
...,...,...,...,...,...,...,...,...
423047,423048,423049,'8A26AB0D-D980-48B0-A2C1-314600566755','M20 J10A','M20','Motorway',0.920421,51.133086
423048,423049,423050,'58F564B3-CEE8-4C7A-9B6D-FB82097DC68C','M20 J11','M20','Motorway',1.050151,51.095487
423049,423050,423051,'F0D6AAC7-6AA9-4F21-A7B9-76253E91FF33','M20 J11A','M20','Motorway',1.099786,51.093398
423050,423051,423052,'B69471D9-B06A-4664-9911-EB1C6191AD93','M20 J12','M20','Motorway',1.131499,51.091885


In [9]:
code_l = {}
name_l = {}
tp_l = {}
dist_l = {}

In [10]:
code_l[0] = []
code_l[1] = []
code_l[2] = []
name_l[0] = []
name_l[1] = []
name_l[2] = []
tp_l[0] = []
tp_l[1] = []
tp_l[2] = []
dist_l[0] = []
dist_l[1] = []
dist_l[2] = []

In [12]:
for x in range(len(df.index)):
    res = tree.query([(df.iloc[x]['LATITUDE'],df.iloc[x]['LONGITUDE'])],k=3)
    ind = res[1][0].tolist()
#     while(len(ind)<3):
#         print("grdoooon")
#         ind.append(0)
        
    for i in range(len(ind)):
        pos = ind[i]
        #print(pos)
        code_l[i].append(dff.iloc[pos]['code'])
        name_l[i].append(dff.iloc[pos]['name'])
        tp_l[i].append(dff.iloc[pos]['type'])
        dsto = my_dist((df.iloc[x]['LATITUDE'],df.iloc[x]['LONGITUDE']),(dff.iloc[pos]['lat'],dff.iloc[pos]['long']))
        dist_l[i].append(dsto)
    if len(code_l[0])!=len(code_l[1]):
        print(x,'nafig')
        break
    if x%20000 == 0:
        print(x)
#         df.loc[x,'code_{}'.format(i)] = dff.iloc[pos]['code']
#         df.loc[x,'name_{}'.format(i)] = dff.iloc[pos]['name']
#         df.loc[x,'type_{}'.format(i)] = dff.iloc[pos]['type']

0
20000
40000
60000


In [13]:
len(dist_l[1])

61123

In [14]:
len(name_l[2])

61123

In [15]:
df['transport_code_0'] = code_l[0]
df['transport_code_1'] = code_l[1]
df['transport_code_2'] = code_l[2]
df['transport_name_0'] = name_l[0]
df['transport_name_1'] = name_l[1]
df['transport_name_2'] = name_l[2]
df['transport_type_0'] = tp_l[0]
df['transport_type_1'] = tp_l[1]
df['transport_type_2'] = tp_l[2]
df['transport_dist_0'] = dist_l[0]
df['transport_dist_1'] = dist_l[1]
df['transport_dist_2'] = dist_l[2]

In [51]:
df.rename(columns = {'transport_dist_1': 'transport_dist_1_km'},inplace=True)

In [15]:
df.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1,inplace=True)

In [17]:
df

,Unnamed: 0,Unnamed: 0.1,Date,Property type,Price,Source,Address,KeyFeatures,latitude,longitude,...,transport_code_2,transport_name_0,transport_name_1,transport_name_2,transport_type_0,transport_type_1,transport_type_2,transport_dist_0,transport_dist_1,transport_dist_2
0,0,0,25 Jun 2021,"Detached, Freehold","£995,000",Land Registry,"41 Runcie Close, St Albans, Hertfordshire AL4 9AX",Detached\nFour Bedrooms\nTwo Bathrooms\nStudy\...,51.770175,-0.313172,...,'210021502070','Pondfield Crescent','Marshalswick Lane','Marshalswick Lane','Bus','Bus','Bus',0.237281,0.299074,0.313605
1,1,1,10 Jun 2021,"Terraced, Freehold","£650,000",Land Registry,"37 Albany Road, Birmingham, West Midlands B17 9JX",VICTORIAN MID TERRACE\nFOUR BEDROOMS\nPERIOD F...,52.459072,-1.950948,...,'43000320202','Harborne Junction','Harborne Junction','Harborne Junction','Bus','Bus','Bus',0.133796,0.137707,0.129067
2,2,2,30 Jun 2021,"Detached, Freehold","£350,000",Land Registry,"19 Bibsworth Avenue, Birmingham, West Midlands...",Detached\nThree Bedrooms\nKitchen Diner\nTwo R...,52.432568,-1.859492,...,'43000429301','Bibsworth Avenue','Wake Green Rd','Thirlmere Drive','Bus','Bus','Bus',0.101797,0.148193,0.215372
3,3,3,18 Jun 2021,"Detached, Freehold","£382,500",Land Registry,"24 Sandy Croft, Birmingham, West Midlands B13 0EP",3 Bedrooms\nPorch\nEntrance Hall\nLounge\nKitc...,52.427717,-1.863037,...,'43000432702','Westridge Rd','Westridge Rd','Dene Hollow','Bus','Bus','Bus',0.097220,0.105499,0.187090
4,4,4,30 Jul 2021,"Semi-Detached, Freehold","£213,500",Land Registry,"15 Poppy Walk, Hatfield, Hertfordshire AL10 9FP",One Bedroomed House\nAllocated Parking\nChain ...,51.774284,-0.237790,...,'210021690570','Daisy Drive','Mulberry Mead','Mulberry Mead','Bus','Bus','Bus',0.112517,0.192354,0.193202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61118,61118,61118,Aug 2021,NaN,"£225,000",NaN,53 Strathmore Avenue LU1 3NY,"Terraced house, Freehold, 3 Beds, 2 Baths, 2 R...",51.872773,-0.408768,...,'2905047','Seymour Avenue','New Town Road','Cambridge Street','Bus','Bus','Bus',0.064839,0.059859,0.070945
61119,61119,61119,Aug 2021,NaN,"£60,000",NaN,13 Robinson Street DH8 5TR,"Terraced house, Freehold, 2 Beds, 1 Bath, 2 Re...",54.861234,-1.841458,...,'13001573G','The Cricketers','The Cricketers','The Cricketers','Bus','Bus','Bus',0.094796,0.177964,0.179140
61120,61120,61120,Aug 2021,NaN,"£125,000",NaN,10 Merrington Road TF2 8PY,"Semi-detached house, Freehold",52.725638,-2.419358,...,'3590E048000','Sutherland Drive','Sutherland Drive','Fieldhouse Drive','Bus','Bus','Bus',0.257807,0.243798,0.186875
61121,61121,61121,Aug 2021,NaN,"£318,000",NaN,"8 Mount View, Church Lane West GU11 3LN",NaN,51.243067,-0.767056,...,'1900HA090300','Bridge Road','Bridge Road','Halimote Road','Bus','Bus','Bus',0.146733,0.175703,0.233379


In [16]:
df.to_csv('ZP_RM_61k_with_address_base_estimated_misses.csv')